# 03_training.ipynb - GITHUB DATA SOURCE VERSION

**This notebook:**
1. Downloads raw NetCDF data directly from GitHub (both `Dataset/` and `2025 data/` folders)
2. Preprocesses and batches the data **in streaming mode** (low memory usage)
3. Trains the WeatherNowcaster model with Mixed Precision (AMP)

**Prerequisites:**
- Enable GPU: Runtime → Change runtime type → T4 GPU

**Expected: ~2-5 min per epoch on T4**

In [ ]:
# Cell 1: Setup & GPU Verification
!pip install -q netCDF4

import os, gc, glob, time, requests, zipfile, io
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from datetime import datetime

# CRITICAL: Verify GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if device.type != 'cuda':
    print('⚠️ WARNING: Running on CPU! Enable GPU: Runtime → Change runtime type → GPU')
else:
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
    torch.backends.cudnn.benchmark = True
    print('✓ cudnn.benchmark enabled')

# Mixed Precision
from torch.cuda.amp import autocast, GradScaler
scaler = GradScaler()
print('✓ Mixed Precision (AMP) enabled')

# Paths
WORK_DIR = '/content/weather_nowcasting'
DATA_DIR = f'{WORK_DIR}/Dataset'
DATA_2025_DIR = f'{WORK_DIR}/2025_data'
BATCHED_DIR = f'{WORK_DIR}/data/batched'
CHECKPOINT_DIR = f'{WORK_DIR}/checkpoints'
FIGURES_DIR = f'{WORK_DIR}/figures'

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(DATA_2025_DIR, exist_ok=True)
os.makedirs(BATCHED_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(FIGURES_DIR, exist_ok=True)

print(f'\nWork directory: {WORK_DIR}')

In [ ]:
# Cell 2: Download Dataset from GitHub
GITHUB_REPO = 'ui07xWizardOp/paper-weather-nowcasting'
BRANCH = 'main'

def download_github_repo():
    """Download the entire repository as a ZIP and extract Dataset + 2025 data folders."""
    zip_url = f'https://github.com/{GITHUB_REPO}/archive/refs/heads/{BRANCH}.zip'
    print(f'Downloading from: {zip_url}')
    
    response = requests.get(zip_url, stream=True)
    if response.status_code != 200:
        raise Exception(f'Failed to download: HTTP {response.status_code}')
    
    total_size = int(response.headers.get('content-length', 0))
    print(f'Download size: {total_size / 1e6:.1f} MB')
    
    chunks = []
    for chunk in tqdm(response.iter_content(chunk_size=8192), 
                      total=total_size//8192, desc='Downloading'):
        chunks.append(chunk)
    
    zip_data = b''.join(chunks)
    print(f'Downloaded {len(zip_data) / 1e6:.1f} MB')
    
    with zipfile.ZipFile(io.BytesIO(zip_data), 'r') as zf:
        print('\nExtracting Dataset folder (2015-2024 data)...')
        dataset_files = [f for f in zf.namelist() if '/Dataset/' in f and f.endswith('.nc')]
        print(f'Found {len(dataset_files)} NetCDF files in Dataset/')
        
        for file in tqdm(dataset_files, desc='Extracting Dataset'):
            filename = os.path.basename(file)
            if filename:
                with zf.open(file) as src:
                    with open(f'{DATA_DIR}/{filename}', 'wb') as dst:
                        dst.write(src.read())
        
        print('\nExtracting 2025 data folder...')
        data_2025_files = [f for f in zf.namelist() if '/2025 data/' in f and f.endswith('.nc')]
        print(f'Found {len(data_2025_files)} NetCDF files in 2025 data/')
        
        for file in tqdm(data_2025_files, desc='Extracting 2025 data'):
            filename = os.path.basename(file)
            if filename:
                with zf.open(file) as src:
                    with open(f'{DATA_2025_DIR}/{filename}', 'wb') as dst:
                        dst.write(src.read())
    
    del zip_data, chunks
    gc.collect()
    
    main_files = glob.glob(f'{DATA_DIR}/*.nc')
    files_2025 = glob.glob(f'{DATA_2025_DIR}/*.nc')
    print(f'\n✓ Extracted {len(main_files)} files to Dataset/')
    print(f'✓ Extracted {len(files_2025)} files to 2025_data/')
    return main_files, files_2025

existing_main = glob.glob(f'{DATA_DIR}/*.nc')
existing_2025 = glob.glob(f'{DATA_2025_DIR}/*.nc')

if len(existing_main) >= 100 and len(existing_2025) >= 10:
    print(f'✓ Dataset already exists:')
    print(f'  - Main (2015-2024): {len(existing_main)} files')
    print(f'  - 2025 data: {len(existing_2025)} files')
else:
    existing_main, existing_2025 = download_github_repo()

print(f'\nTotal NetCDF files: {len(existing_main) + len(existing_2025)}')

In [ ]:
# Cell 3: Memory-Efficient Preprocessing Configuration
import xarray as xr
import pandas as pd

# Configuration
T_IN = 24
T_OUT = 6
STRIDE = 6
SAMPLES_PER_BATCH = 500
VARIABLES = ['tp', 't2m']

TRAIN_YEARS = list(range(2015, 2022))
VAL_YEARS = list(range(2022, 2024))
TEST_YEARS = list(range(2024, 2027))

def load_single_file(filepath):
    """Load a NetCDF file, handling different coordinate naming conventions."""
    # Try different engines
    for engine in ['netcdf4', 'h5netcdf', 'scipy', None]:
        try:
            if engine:
                ds = xr.open_dataset(filepath, engine=engine)
            else:
                ds = xr.open_dataset(filepath)
            break
        except Exception:
            continue
    else:
        raise ValueError(f'Could not open {filepath} with any engine')
    
    # Handle 'valid_time' vs 'time' naming
    if 'valid_time' in ds.coords and 'time' not in ds.coords:
        ds = ds.rename({'valid_time': 'time'})
    
    # Handle 'expver' dimension (experimental version)
    if 'expver' in ds.dims:
        ds = ds.isel(expver=0, drop=True)
    elif 'expver' in ds.coords:
        ds = ds.drop_vars('expver', errors='ignore')
    
    # Handle 'number' coordinate
    if 'number' in ds.coords:
        ds = ds.drop_vars('number', errors='ignore')
    
    return ds

# Check if batched data already exists
batched_train = glob.glob(f'{BATCHED_DIR}/train/X_batch_*.npy')
if len(batched_train) > 0:
    print(f'✓ Batched data already exists: {len(batched_train)} train batches')
    print('  Skipping preprocessing...')
    PREPROCESSING_DONE = True
else:
    PREPROCESSING_DONE = False
    print('Will run memory-efficient preprocessing...')

In [ ]:
# Cell 4: MEMORY-EFFICIENT Preprocessing (Streaming per-file)
if not PREPROCESSING_DONE:
    all_nc_files = sorted(glob.glob(f'{DATA_DIR}/*.nc')) + sorted(glob.glob(f'{DATA_2025_DIR}/*.nc'))
    print(f'Total files to process: {len(all_nc_files)}')
    
    # Helper: extract variables from dataset
    def extract_vars(ds):
        var_data = []
        for var in VARIABLES:
            if var in ds.data_vars:
                var_data.append(ds[var].values)
            elif var == 't2m' and '2m_temperature' in ds.data_vars:
                var_data.append(ds['2m_temperature'].values)
            elif var == 'tp' and 'total_precipitation' in ds.data_vars:
                var_data.append(ds['total_precipitation'].values)
        if len(var_data) == len(VARIABLES):
            return np.stack(var_data, axis=-1).astype(np.float32)
        return None
    
    # ========== PASS 1: Compute normalization stats from training data ==========
    print('\n[Pass 1] Computing normalization statistics...')
    train_values = []
    files_loaded = 0
    
    for f in tqdm(all_nc_files, desc='Scanning for stats'):
        try:
            ds = load_single_file(f)
            year = pd.to_datetime(ds.time.values[0]).year
            
            if year in TRAIN_YEARS:
                data = extract_vars(ds)
                if data is not None:
                    # Sample every 24th hour to reduce memory
                    train_values.append(data[::24])
                    files_loaded += 1
            ds.close()
        except Exception as e:
            print(f'Warning: {os.path.basename(f)}: {e}')
    
    print(f'Loaded stats from {files_loaded} training files')
    
    if not train_values:
        raise ValueError('No training data found! Check file formats.')
    
    train_sample = np.concatenate(train_values, axis=0)
    mean = np.nanmean(train_sample, axis=(0, 1, 2))
    std = np.nanstd(train_sample, axis=(0, 1, 2))
    std[std < 1e-6] = 1.0
    print(f'Mean: {mean}')
    print(f'Std: {std}')
    del train_values, train_sample
    gc.collect()
    
    # ========== PASS 2: Process files and create sequences ==========
    print('\n[Pass 2] Processing files and creating sequences...')
    
    for split in ['train', 'val', 'test']:
        os.makedirs(f'{BATCHED_DIR}/{split}', exist_ok=True)
    
    # Accumulators
    X_buffers = {'train': [], 'val': [], 'test': []}
    Y_buffers = {'train': [], 'val': [], 'test': []}
    batch_counters = {'train': 0, 'val': 0, 'test': 0}
    
    def save_buffer(split):
        """Save accumulated buffer to disk."""
        if len(X_buffers[split]) >= SAMPLES_PER_BATCH:
            X_batch = np.stack(X_buffers[split][:SAMPLES_PER_BATCH])
            Y_batch = np.stack(Y_buffers[split][:SAMPLES_PER_BATCH])
            np.save(f'{BATCHED_DIR}/{split}/X_batch_{batch_counters[split]:04d}.npy', X_batch)
            np.save(f'{BATCHED_DIR}/{split}/Y_batch_{batch_counters[split]:04d}.npy', Y_batch)
            batch_counters[split] += 1
            X_buffers[split] = X_buffers[split][SAMPLES_PER_BATCH:]
            Y_buffers[split] = Y_buffers[split][SAMPLES_PER_BATCH:]
    
    def create_sequences(data, t_in, t_out):
        """Create sequences from contiguous data."""
        X_list, Y_list = [], []
        n = len(data)
        if n < t_in + t_out:
            return [], []
        for i in range(t_in, n - t_out + 1):
            X_list.append(data[i-t_in:i])
            Y_list.append(data[i:i+t_out])
        return X_list, Y_list
    
    # Process each file
    for f in tqdm(all_nc_files, desc='Processing files'):
        try:
            ds = load_single_file(f)
            data = extract_vars(ds)
            
            if data is None:
                ds.close()
                continue
            
            # Get year from first timestamp
            year = pd.to_datetime(ds.time.values[0]).year
            ds.close()
            
            # Normalize
            data = np.nan_to_num(data, nan=0.0)
            data = (data - mean) / std
            
            # Create sequences
            X_seqs, Y_seqs = create_sequences(data, T_IN, T_OUT)
            
            # Determine split
            if year in TRAIN_YEARS:
                split = 'train'
            elif year in VAL_YEARS:
                split = 'val'
            elif year in TEST_YEARS:
                split = 'test'
            else:
                continue
            
            # Add to buffer
            X_buffers[split].extend(X_seqs)
            Y_buffers[split].extend(Y_seqs)
            
            # Save if buffer full
            while len(X_buffers[split]) >= SAMPLES_PER_BATCH:
                save_buffer(split)
            
            del data, X_seqs, Y_seqs
        
        except Exception as e:
            print(f'Warning: {os.path.basename(f)}: {e}')
    
    # Final flush - save remaining sequences
    for split in ['train', 'val', 'test']:
        if X_buffers[split]:
            X_batch = np.stack(X_buffers[split])
            Y_batch = np.stack(Y_buffers[split])
            np.save(f'{BATCHED_DIR}/{split}/X_batch_{batch_counters[split]:04d}.npy', X_batch)
            np.save(f'{BATCHED_DIR}/{split}/Y_batch_{batch_counters[split]:04d}.npy', Y_batch)
            batch_counters[split] += 1
    
    # Save stats
    np.savez(f'{BATCHED_DIR}/stats.npz', mean=mean, std=std, variables=np.array(VARIABLES))
    
    print(f'\n✓ Preprocessing complete!')
    print(f'  Train: {batch_counters["train"]} batches')
    print(f'  Val: {batch_counters["val"]} batches')
    print(f'  Test: {batch_counters["test"]} batches')
    
    gc.collect()
else:
    print('Using existing batched data.')

In [ ]:
# Cell 5: Data Loading Utilities
BATCH_SIZE = 32

def get_batch_files(split):
    split_dir = f'{BATCHED_DIR}/{split}'
    x_files = sorted(glob.glob(f'{split_dir}/X_batch_*.npy'))
    y_files = sorted(glob.glob(f'{split_dir}/Y_batch_*.npy'))
    return x_files, y_files

def batch_generator(split, batch_size=BATCH_SIZE, shuffle=False):
    x_files, y_files = get_batch_files(split)
    file_indices = list(range(len(x_files)))
    if shuffle:
        np.random.shuffle(file_indices)
    
    for idx in file_indices:
        X = np.load(x_files[idx])
        Y = np.load(y_files[idx])
        n = len(X)
        indices = np.random.permutation(n) if shuffle else np.arange(n)
        
        for start in range(0, n, batch_size):
            batch_idx = indices[start:start+batch_size]
            x = torch.from_numpy(X[batch_idx]).float().permute(0, 1, 4, 2, 3)
            y = torch.from_numpy(Y[batch_idx]).float().permute(0, 1, 4, 2, 3)
            yield x, y

# Load stats
stats = np.load(f'{BATCHED_DIR}/stats.npz', allow_pickle=True)
mean, std = stats['mean'], stats['std']
variables = list(stats['variables'])
print(f'Variables: {variables}')
print(f'Mean: {mean}')
print(f'Std: {std}')

# Count batches
train_files, _ = get_batch_files('train')
val_files, _ = get_batch_files('val')
test_files, _ = get_batch_files('test')

SAMPLES_PER_FILE = SAMPLES_PER_BATCH
BATCHES_PER_FILE = (SAMPLES_PER_FILE + BATCH_SIZE - 1) // BATCH_SIZE
n_train_batches = len(train_files) * BATCHES_PER_FILE
n_val_batches = len(val_files) * BATCHES_PER_FILE
n_test_batches = len(test_files) * BATCHES_PER_FILE

print(f'\nTrain: {len(train_files)} files = ~{n_train_batches} batches')
print(f'Val: {len(val_files)} files = ~{n_val_batches} batches')
print(f'Test: {len(test_files)} files = ~{n_test_batches} batches')

# Quick benchmark
print('\nBenchmarking data loading...')
t0 = time.time()
for i, (x, y) in enumerate(batch_generator('train')):
    if i >= 20:
        break
print(f'20 batches in {time.time()-t0:.2f}s ({(time.time()-t0)/20*1000:.1f}ms/batch)')

In [ ]:
# Cell 6: Model Definition
class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.conv = nn.Conv2d(input_dim + hidden_dim, 4 * hidden_dim,
                              kernel_size, padding=kernel_size//2)
    def forward(self, x, hidden):
        h, c = hidden
        gates = self.conv(torch.cat([x, h], dim=1))
        i, f, o, g = torch.chunk(gates, 4, dim=1)
        c_next = torch.sigmoid(f) * c + torch.sigmoid(i) * torch.tanh(g)
        h_next = torch.sigmoid(o) * torch.tanh(c_next)
        return h_next, c_next
    def init_hidden(self, B, H, W, dev):
        return (torch.zeros(B, self.hidden_dim, H, W, device=dev),
                torch.zeros(B, self.hidden_dim, H, W, device=dev))

class WeatherNowcaster(nn.Module):
    def __init__(self, in_ch, hidden_dim, out_ch, n_layers=2):
        super().__init__()
        self.encoder = nn.ModuleList([ConvLSTMCell(in_ch if i==0 else hidden_dim, hidden_dim, 3) for i in range(n_layers)])
        self.decoder = nn.ModuleList([ConvLSTMCell(out_ch if i==0 else hidden_dim, hidden_dim, 3) for i in range(n_layers)])
        self.out_conv = nn.Conv2d(hidden_dim, out_ch, 1)
    
    def forward(self, x, future_steps):
        B, T, C, H, W = x.shape
        hidden = [cell.init_hidden(B, H, W, x.device) for cell in self.encoder]
        for t in range(T):
            inp = x[:, t]
            for i, cell in enumerate(self.encoder):
                h, c = cell(inp, hidden[i])
                hidden[i] = (h, c)
                inp = h
        
        dec_hidden = [(h.clone(), c.clone()) for h, c in hidden]
        outputs = []
        dec_in = self.out_conv(dec_hidden[-1][0])
        for _ in range(future_steps):
            for i, cell in enumerate(self.decoder):
                h, c = cell(dec_in if i==0 else h, dec_hidden[i])
                dec_hidden[i] = (h, c)
            dec_in = self.out_conv(h)
            outputs.append(dec_in)
        return torch.stack(outputs, dim=1)

HIDDEN_DIM = 128
IN_CHANNELS = len(variables)
OUT_CHANNELS = len(variables)

model = WeatherNowcaster(IN_CHANNELS, HIDDEN_DIM, OUT_CHANNELS, n_layers=2).to(device)
print(f'Model Parameters: {sum(p.numel() for p in model.parameters()):,}')

# Benchmark forward pass
print('\nBenchmarking model...')
for x, y in batch_generator('train'):
    _, _, C, H, W = x.shape
    break

x_test = torch.randn(BATCH_SIZE, T_IN, IN_CHANNELS, H, W).to(device)
torch.cuda.synchronize()
t0 = time.time()
for _ in range(10):
    with autocast():
        _ = model(x_test, T_OUT)
    torch.cuda.synchronize()
print(f'10 forward passes in {time.time()-t0:.2f}s ({(time.time()-t0)/10*1000:.1f}ms/batch)')
del x_test
torch.cuda.empty_cache()

In [ ]:
# Cell: Auto-Resume Logic & Drive Mounting
import os
import torch
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print('✓ Google Drive mounted')
    CHECKPOINT_DIR = '/content/drive/MyDrive/WeatherPaper/checkpoints'
    os.makedirs(CHECKPOINT_DIR, exist_ok=True)
    print(f'✓ Checkpoints will be saved to: {CHECKPOINT_DIR}')
except ImportError:
    print('⚠️ Not running in Colab, using local paths')
    CHECKPOINT_DIR = 'checkpoints'

RESUME_FROM = None
start_epoch = 0

# Check for existing LAST checkpoint
if os.path.exists(f'{CHECKPOINT_DIR}/last.pth'):
    RESUME_FROM = f'{CHECKPOINT_DIR}/last.pth'
elif os.path.exists(f'{CHECKPOINT_DIR}/best_model.pth'):
    RESUME_FROM = f'{CHECKPOINT_DIR}/best_model.pth'

if RESUME_FROM and os.path.exists(RESUME_FROM):
    print(f'Loading checkpoint: {RESUME_FROM}')
    checkpoint = torch.load(RESUME_FROM, map_location='cuda' if torch.cuda.is_available() else 'cpu')
    if 'model' in checkpoint:
        model.load_state_dict(checkpoint['model'])
    else:
        model.load_state_dict(checkpoint)
    
    if 'epoch' in checkpoint:
        start_epoch = checkpoint['epoch'] + 1
        print(f'Resuming from epoch {start_epoch}')
    else:
        print('Warning: Epoch info not found, fine-tuning from 0')


In [ ]:
# Cell 7: Training Loop with AMP
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)

NUM_EPOCHS = 50
best_val_loss = float('inf')
train_losses, val_losses = [], []

print(f'Training Configuration:')
print(f'  Epochs: {NUM_EPOCHS}')
print(f'  Batch size: {BATCH_SIZE}')
print(f'  Hidden dim: {HIDDEN_DIM}')
print(f'  Input: {T_IN}h → Output: {T_OUT}h')
print('=' * 60)

for epoch in range(start_epoch, NUM_EPOCHS):
    epoch_start = time.time()
    
    model.train()
    train_loss, n_batches = 0.0, 0
    pbar = tqdm(batch_generator('train', BATCH_SIZE, shuffle=True), 
                total=n_train_batches, desc=f'Epoch {epoch+1} [Train]', leave=False)
    
    for x, y in pbar:
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        
        with autocast():
            out = model(x, T_OUT)
            loss = criterion(out, y)
        
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()
        n_batches += 1
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    train_loss /= n_batches
    train_losses.append(train_loss)
    
    model.eval()
    val_loss, n_val = 0.0, 0
    with torch.no_grad():
        for x, y in tqdm(batch_generator('val', BATCH_SIZE), total=n_val_batches, 
                         desc=f'Epoch {epoch+1} [Val]', leave=False):
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            with autocast():
                val_loss += criterion(model(x, T_OUT), y).item()
            n_val += 1
    
    val_loss /= n_val
    val_losses.append(val_loss)
    scheduler.step(val_loss)
    
    epoch_time = time.time() - epoch_start
    marker = '★ BEST' if val_loss < best_val_loss else ''
    print(f'Epoch {epoch+1:2d} | Train: {train_loss:.6f} | Val: {val_loss:.6f} | Time: {epoch_time:.1f}s {marker}')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({'model': model.state_dict(), 'val_loss': val_loss}, 
                   f'{CHECKPOINT_DIR}/best_model.pth')
    
    if (epoch + 1) % 10 == 0:
        torch.save({'epoch': epoch, 'model': model.state_dict(),
                    'train_losses': train_losses, 'val_losses': val_losses},
                   f'{CHECKPOINT_DIR}/checkpoint_e{epoch+1}.pth')
    
    gc.collect()
    torch.cuda.empty_cache()

print('=' * 60)
print(f'✓ Training complete! Best val loss: {best_val_loss:.6f}')

In [ ]:
# Cell 8: Plot Training Curves
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train', linewidth=2)
plt.plot(val_losses, label='Val', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Training Progress')
plt.legend()
plt.grid(alpha=0.3)
plt.savefig(f'{FIGURES_DIR}/training_curve.png', dpi=150, bbox_inches='tight')
plt.show()
print(f'Best val loss: {best_val_loss:.6f}')

In [ ]:
# Cell 9: Sample Prediction
model.eval()
for x, y in batch_generator('val'):
    x_sample = x[:1].to(device)
    y_sample = y[:1]
    break

with torch.no_grad():
    with autocast():
        pred = model(x_sample, T_OUT)

pred_np = pred[0].cpu().float().numpy()
true_np = y_sample[0].numpy()

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
for i, t in enumerate([0, 2, 5]):
    axes[0, i].imshow(true_np[t, 0], cmap='Blues')
    axes[0, i].set_title(f'Truth t+{t+1}h')
    axes[1, i].imshow(pred_np[t, 0], cmap='Blues')
    axes[1, i].set_title(f'Pred t+{t+1}h')
plt.suptitle('Precipitation Forecast', fontsize=14)
plt.tight_layout()
plt.savefig(f'{FIGURES_DIR}/sample_prediction.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Cell 10: Save Final Model
torch.save({
    'model_state_dict': model.state_dict(),
    'config': {'hidden_dim': HIDDEN_DIM, 'n_layers': 2, 'T_IN': T_IN, 'T_OUT': T_OUT},
    'mean': mean, 'std': std, 'variables': variables,
    'best_val_loss': best_val_loss,
    'train_losses': train_losses, 'val_losses': val_losses
}, f'{CHECKPOINT_DIR}/final_model.pth')

print(f'✓ Saved to: {CHECKPOINT_DIR}/final_model.pth')
print(f'Best validation loss: {best_val_loss:.6f}')

In [ ]:
# Cell 11: Test on 2025 Data
print('Evaluating on test set (includes 2025 data)...')
model.eval()
test_loss, n_test = 0.0, 0

with torch.no_grad():
    for x, y in tqdm(batch_generator('test', BATCH_SIZE), total=n_test_batches, desc='Testing'):
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        with autocast():
            test_loss += criterion(model(x, T_OUT), y).item()
        n_test += 1

test_loss /= n_test if n_test > 0 else 1
print(f'\n✓ Test Loss (2024-2025 data): {test_loss:.6f}')